## 5) What attacking IPs performed an Internet wide scan/attack? 
a. What criteria did you use to identify these IPs? Explain.  
b. What attack method(s) did these threat actors use? Explain.  

**Answer**  

#### Server description

- Snort is not a honeypot but an IDS/IPS and is used to detect attacks.
- Dionaea is a low interaction honeypot that exposes services such as MSSQL, SIP, HTTP, FTP, TFTP, SFTP, SSH, and SMB.
- Cowrie is a medium interaction SSH honeypot designed to log brute force attacks and shell interactions performed by the attackers.
- Elastishoney is a simple elasticsearch honeypot designed to catch attackers exploiting remote code execution (RCE) vulnerabilities in elasticsearch.
- Glastopf is a Python based Web application honeypot that has the ability to emulate thousands of web vulnerabilities.
- Wordpot is a WordPress emulator honeypot.
- ShockPot is a web application honeypot that exposes the vulnerability CVE-2014-6271
***
- 192.168.10.2 - dionaea
- 192.168.10.5 - cowrie
- 192.168.10.6 - elastic
- 192.168.10.6 - glastopf
- 192.168.10.3 - wordpot
- 192.168.10.4 - shockpot

REFERENCE: https://stackoverflow.com/questions/45080828/using-pd-grouper-to-group-by-n-minutes-each for grouper solution

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# loading all files
df_raw = pd.read_csv("sorted-AllTraffic.csv",sep='\t', lineterminator='\n')

/Users/dee/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df_raw["asn\r"]=df_raw["asn\r"].str.slice(stop=-1)
df_raw.timestamp = pd.to_datetime(df_raw.timestamp)

In [3]:
df_raw.sample()

,id,ident,normalized,timestamp,unixTimestamp,cal_date,week day,year,month,day,hour,minutes,seconds,channel,source_ip,source_ip_num,source_port,destination_ip,destination_ip_num,destination_port,protocol,city,province,province_code,country,country_code,continent,latitude,longitude,asn
1128931,58ea954593e0950608bd6959,c62a8c76-5ce0-11e6-94de-0800272a9814,True,2017-04-09 16:10:45.531,1.491754e+09,2017-04-09,Sun,2017,4,9,16,10,45.531,dionaea.connections,69.64.43.122,1161833338,42392,192.168.10.2,3232238082,25974,pcap,St Louis,Missouri,MO,United States,US,North America,38.6312,-90.1922,HEG US Inc.


In [4]:
df_raw.loc[(df_raw['channel'] == "snort.alerts") & (df_raw['destination_ip'] == "192.168.10.2"),'channel'] = "dionaea.connections"
df_raw.loc[(df_raw['channel'] == "snort.alerts") & (df_raw['destination_ip'] == "192.168.10.3"),'channel'] = "wordpot.events"
df_raw.loc[(df_raw['channel'] == "snort.alerts") & (df_raw['destination_ip'] == "192.168.10.4"),'channel'] = "shockpot.events"
df_raw.loc[(df_raw['channel'] == "snort.alerts") & (df_raw['destination_ip'] == "192.168.10.5"),'channel'] = "cowrie.sessions"
df_raw.loc[(df_raw['channel'] == "snort.alerts") & (df_raw['destination_ip'] == "192.168.10.6"),'channel'] = "glastopf.events"

In [5]:
df=df_raw[['timestamp', 'channel', 'source_ip',
       'destination_ip', 'destination_port', 'protocol',
       'city', 'country', 'asn\r']]

***

In [6]:
# groupby by frequency, combining connections and count
df_ipfq = df.groupby([pd.Grouper(key='timestamp', freq='2D'),'source_ip'])['channel'].agg([np.unique,'count']).reset_index()

In [7]:
# saving ips with attacks in more than one channel to separate DF
df_ip_multi = df_ipfq[df_ipfq['unique'].map(lambda x:type(x).__module__ == 'numpy')] # multi channel attacks
df_ip_single = df_ipfq[df_ipfq['unique'].map(lambda x:type(x).__module__ != 'numpy')] # single channel attacks

In [8]:
pd.set_option('max_colwidth',100)

# return attacks that where spread in multiple channels
df_ip_multi[(df_ip_multi['unique'].map(len) == 5) & (df_ip_multi['count'] > 500)].sort_values(by=['count'],ascending=False).head(10)

,timestamp,source_ip,unique,count
156086,2017-02-19 16:29:34.636,85.17.29.79,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",8612
157940,2017-02-21 16:29:34.636,85.17.29.79,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",3373
224414,2017-05-24 16:29:34.636,23.248.219.51,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",2472
229983,2017-06-01 16:29:34.636,23.248.219.51,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1932
91082,2016-12-23 16:29:34.636,134.19.176.138,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1508
220820,2017-05-20 16:29:34.636,23.248.219.51,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1359
233021,2017-06-05 16:29:34.636,23.248.219.51,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1342
244850,2017-06-25 16:29:34.636,162.209.168.14,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1246
227614,2017-05-28 16:29:34.636,23.248.219.51,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1230
222632,2017-05-22 16:29:34.636,23.248.219.51,"[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]",1151


***

5) What attacking IPs performed an Internet wide scan/attack?   
a. What criteria did you use to identify these IPs? Explain.  
b. What attack method(s) did these threat actors use? Explain.



First have to convert the snort entries back to its respective servers. Althought snort is an IDS/IPS the attack did occur so we want those attack be taken account back to their respective honeypots. Next I've resampling all data based frequency, grouped all IP within the day range, then performed a count on the total and the honeypots/sensors that has been attacked by that IP. 

The first criteria is to find IP address that has attacked multiple honeypots within a short period of time. 2 day period in at least 5 datafiles.  
1. 2017-02-19	85.17.29.79	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	8612 attacks  
2. 2017-02-21	85.17.29.79	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	3373 attacks  
3. 2017-05-24	23.248.219.51	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	2472 attacks  
4. 2017-06-01	23.248.219.51	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1932 attacks  
5. 2016-12-23	134.19.176.138	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1508 attacks  
6. 2017-05-20	23.248.219.51	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1359 attacks  
7. 2017-06-05	23.248.219.51	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1342 attacks  
8. 2017-06-25	162.209.168.14	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1246 attacks  
9. 2017-05-28	23.248.219.51	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1230 attacks  
10. 2017-05-22	23.248.219.51	[cowrie.sessions, dionaea.connections, glastopf.events, shockpot.events, wordpot.events]	1151 attacks  

IP 85.17.29.79 (#1) and 23.248.219.51 (#3) has clearly launched an internet attack. The attacker's IP address showned appeared multiple times even in my top 10 attack list which means the attack happened in multiple 2-day periods while each of these attacks hint multiple server.

The other IPs there were in the top 10 list were 134.19.176.138 and 162.209.168.14

In [9]:
# An example of attack from IP 85.17.29.79 that span over 2 days.
df[df.source_ip == "85.17.29.79"]

,timestamp,channel,source_ip,destination_ip,destination_port,protocol,city,country,asn
1024717,2017-02-20 07:33:23.363,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1024718,2017-02-20 07:33:24.039,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1024719,2017-02-20 07:33:24.195,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1024720,2017-02-20 07:33:25.031,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1024721,2017-02-20 07:33:25.187,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1024722,2017-02-20 07:33:25.646,wordpot.events,85.17.29.79,192.168.10.3,5900,TCP,None,Netherlands,LeaseWeb Netherlands B.V.
1024723,2017-02-20 07:33:25.873,dionaea.connections,85.17.29.79,192.168.10.2,5900,TCP,None,Netherlands,LeaseWeb Netherlands B.V.
1024724,2017-02-20 07:33:26.071,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1024725,2017-02-20 07:33:26.227,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
1025203,2017-02-20 12:19:36.407,dionaea.connections,85.17.29.79,192.168.10.2,5900,pcap,None,Netherlands,LeaseWeb Netherlands B.V.
